# 1. Refractor script
Bạn được giao nhiệm vụ cải tiến một hệ thống giỏ hàng trực tuyến. Đoạn mã hiện tại bao gồm nhiều hàm độc lập xử lý các tính năng khác nhau cho sản phẩm và giỏ hàng, nhưng nó chưa được tổ chức rõ ràng. Nhiệm vụ của bạn là tái cấu trúc đoạn mã này bằng cách sử dụng các lớp (class) để mã dễ bảo trì và dễ mở rộng.

## Mã ban đầu
Dưới đây là đoạn mã hiện tại, bao gồm các hàm xử lý giảm giá, thuế, quản lý giỏ hàng, và kiểm tra hàng tồn kho. Mỗi sản phẩm là một từ điển (dictionary) với các thuộc tính như tên, giá, và số lượng hàng tồn kho.

In [ ]:
# Dữ liệu mẫu
products = [
    {'name': 'Book', 'price': 15.0, 'stock': 10},
    {'name': 'Food', 'price': 8.0, 'stock': 20},
    {'name': 'Gadget', 'price': 25.0, 'stock': 5}
]

# Các hàm hiện tại
def apply_discount(product, discount):
    return product['price'] * (1 - discount)

def calculate_tax(product, tax_rate):
    return product['price'] * tax_rate

def calculate_total(cart):
    total = 0
    for item in cart:
        total += item['price']
    return total

def add_product(cart, product):
    cart.append(product)

def remove_product(cart, product_name):
    cart[:] = [p for p in cart if p['name'] != product_name]

def list_products(cart):
    for product in cart:
        print(f"{product['name']}: ${product['price']}")

def update_stock(product, amount):
    product['stock'] += amount

def is_in_stock(product):
    return product['stock'] > 0

def get_total_stock(products):
    return sum(product['stock'] for product in products)


## Yêu cầu

### 1. Tái cấu trúc mã: Sử dụng các lớp để tổ chức lại mã. (30%)

* Product: Lớp này sẽ đại diện cho một sản phẩm và chứa các phương thức liên quan đến sản phẩm.
* Cart: Lớp này sẽ quản lý các sản phẩm trong giỏ hàng và chứa các phương thức để thêm, xóa sản phẩm và tính tổng giá.

### 2. Cập nhật mã (30%)
Di chuyển các hàm hiện tại vào các lớp thích hợp và điều chỉnh lại mã để sử dụng các phương thức thay vì các hàm độc lập.

### 3. Xây dựng ví dụ: Sau khi tổ chức lại mã, tạo một ví dụ tuỳ ý để hiển thị (15%)

* Thêm sản phẩm vào giỏ hàng.
* Tính tổng giá sau khi áp dụng thuế và giảm giá.
* In danh sách sản phẩm trong giỏ hàng.


In [ ]:
class Product:
    def __init__(self, name, price, stock):
        """Khởi tạo một sản phẩm với tên, giá và số lượng hàng tồn kho."""
        self.name = name
        self.price = price
        self.stock = stock

    def apply_discount(self, discount):
        """Áp dụng giảm giá cho sản phẩm."""
        return self.price * (1 - discount)

    def calculate_tax(self, tax_rate):
        """Tính thuế cho sản phẩm."""
        return self.price * tax_rate

    def is_in_stock(self, quantity=1):
        """Kiểm tra xem sản phẩm có đủ hàng trong kho không."""
        return self.stock >= quantity

    def update_stock(self, quantity):
        """Giảm số lượng hàng tồn kho khi đặt hàng."""
        if quantity <= self.stock:
            self.stock -= quantity
        else:
            raise ValueError("Số lượng không đủ trong kho!")

    def restock(self, quantity):
        """Tăng số lượng hàng tồn kho khi sản phẩm được trả lại."""
        self.stock += quantity


class Cart:
    def __init__(self):
        """Khởi tạo một giỏ hàng trống."""
        self.items = []

    def add_product(self, product, quantity=1):
        """Thêm sản phẩm vào giỏ hàng với số lượng chỉ định."""
        if product.is_in_stock(quantity):
            self.items.append({'product': product, 'quantity': quantity})
            product.update_stock(quantity)
        else:
            print(f"{product.name} không đủ hàng trong kho cho {quantity} sản phẩm!")

    def remove_quantity(self, product_name, quantity=1):
        """Giảm một số lượng sản phẩm trong giỏ hàng và cập nhật lại kho hàng."""
        for item in self.items:
            if item['product'].name == product_name:
                if item['quantity'] > quantity:
                    item['quantity'] -= quantity
                    item['product'].restock(quantity)
                else:
                    # Nếu số lượng muốn xóa >= số lượng hiện tại, xóa sản phẩm khỏi giỏ hàng
                    item['product'].restock(item['quantity'])
                    self.items.remove(item)
                break

    def calculate_total(self):
        """Tính tổng giá các sản phẩm trong giỏ."""
        return sum(item['product'].price * item['quantity'] for item in self.items)

    def calculate_total_with_discount_and_tax(self, discount, tax_rate):
        """Tính tổng giá sau khi áp dụng giảm giá và thuế."""
        total = 0
        for item in self.items:
            product = item['product']
            quantity = item['quantity']
            discounted_price = product.apply_discount(discount) * quantity
            total += discounted_price + product.calculate_tax(tax_rate) * quantity
        return total

    def list_products(self):
        """In danh sách các sản phẩm trong giỏ hàng."""
        for item in self.items:
            product = item['product']
            quantity = item['quantity']
            print(f"{product.name} x {quantity}: ${product.price * quantity:.2f}")


# Dữ liệu mẫu sản phẩm
products_data = [
    {'name': 'Book', 'price': 15.0, 'stock': 10},
    {'name': 'Food', 'price': 8.0, 'stock': 20},
    {'name': 'Gadget', 'price': 25.0, 'stock': 5}
]

# Tạo các sản phẩm từ dữ liệu mẫu
products = [Product(**data) for data in products_data]

# Tạo giỏ hàng và thêm sản phẩm
cart = Cart()
cart.add_product(products[0], quantity=3)  # Thêm x3 Book
cart.add_product(products[1], quantity=5)  # Thêm x5 Food
cart.add_product(products[2], quantity=6)  # Thêm x6 Gadget

# In danh sách sản phẩm trong giỏ hàng và tổng giá ban đầu
print("Giỏ hàng ban đầu:")
cart.list_products()
print(f"Tổng giá ban đầu: ${cart.calculate_total():.2f}")

# Loại bỏ x1 Book khỏi giỏ hàng
cart.remove_quantity("Book", quantity=1)

# Áp dụng giảm giá và thuế sau khi cập nhật giỏ hàng
discount = 0.2  # Giảm giá 20%
tax = 0.06  # Thuế 6%
total_with_discount_and_tax = cart.calculate_total_with_discount_and_tax(discount, tax)

# In giỏ hàng và tổng giá sau khi loại bỏ x1 Book
print("\nGiỏ hàng sau khi trả lại x1 Book:")
cart.list_products()
print(f"Tổng giá sau khi áp dụng giảm giá và thuế: ${total_with_discount_and_tax:.2f}")


# 2. Viết hàm từ flowchart sau (20%)
Link ảnh: https://drive.google.com/file/d/1nLY0RH4GrA6OXEuDqhIOGMmVPikvmMkL/view?usp=sharing

In [3]:
def f(n):
    a = 0
    b = 1
    count = 2
    
    print(a)
    print(b)
    
    while count <= n:
        next = a + b
        
        print(next)
        
        a = b
        b = next
        count += 1 

n = int(input("Nhập giá trị của n: "))
f(n)


0 1 1 2 3 5 

## Cho biết tác dụng của thuật toán trong flowchart (5%)